# Parallel Simulations for Prediction of aging parameters for partial data window

In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)
from joblib import Parallel, delayed, parallel_backend

eSOH_DIR = "../data/esoh/"
oCV_DIR = "../data/ocv/"
fig_DIR = "../figures/figures_fit/"
res_DIR = "../data/results_fit/"
# %matplotlib widget

In [2]:
parameter_values = get_parameter_values()

In [3]:
parameter_values.search("Li")

Cell cooling surface area [m2]	0.41
Initial plated lithium concentration [mol.m-3]	0.0
Inner SEI lithium interstitial diffusivity [m2.s-1]	1e-20
Lithium interstitial reference concentration [mol.m-3]	15.0
Lithium metal partial molar volume [m3.mol-1]	1.3e-05
Typical lithium ion diffusivity [m2.s-1]	5.34e-10
Typical plated lithium concentration [mol.m-3]	1000.0


In [7]:
cell = 1

## Load eSOH Data and OCV Data

In [8]:
cell_no = f'{cell:02d}'
dfe=pd.read_csv(eSOH_DIR+"aging_param_cell_"+cell_no+".csv")
# Remove first RPT
dfe = dfe.drop(dfe.index[0])
dfe = dfe.reset_index(drop=True)
dfe['N']=dfe['N']-dfe['N'][0]
N =dfe.N.unique()
dfo_0=pd.read_csv(oCV_DIR+"ocv_data_cell_"+cell_no+".csv")
print("Cycle Numbers:")
print(*N, sep = ", ") 

Cycle Numbers:
0, 18, 57, 93, 134, 175, 216, 257, 298, 339


In [9]:
dfo = dfo_0[dfo_0['N']==N[0]]
# dfe_0 = dfe[dfe['N']==N[0]]
C_n_init = dfe['C_n'][0]
C_p_init = dfe['C_p'][0]
y_0_init = dfe['y_0'][0] 
if cell_no=='01':
    c_rate_c = 'C/5'
    c_rate_d = 'C/5'
    dis_set = " until 3V"
    # eps_n_data = 0.6234
    # eps_p_data = 0.4542
elif cell_no=='13':
    c_rate_c = 'C/5'
    c_rate_d = 'C/5'
    dis_set = " for 150 min"

In [10]:
dfe.N.iloc[-1]

339

In [11]:
pybamm.set_logging_level("WARNING")
# pybamm.set_logging_level("NOTICE")
experiment = pybamm.Experiment(
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/50")
    ] *dfe.N.iloc[-1],
    termination="50% capacity",
#     cccv_handling="ode",
)
spm = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        # "loss of active material": ("stress-driven","none"),
        "loss of active material": "stress-driven",
        "stress-induced diffusion": "true",
        # "lithium plating": "reversible",
    }
)
# spm.print_parameter_info()

In [12]:
param = spm.param
eps_n_data = parameter_values.evaluate(C_n_init*3600/(param.n.L * param.n.c_max * param.F* param.A_cc))
eps_p_data = parameter_values.evaluate(C_p_init*3600/(param.p.L * param.p.c_max * param.F* param.A_cc))
cs_p_init = parameter_values.evaluate(y_0_init* param.p.c_max)

In [13]:
parameter_values = get_parameter_values()
sim_des = '_lowc'
parameter_values.update(
    {
        "SEI kinetic rate constant [m.s-1]": 1.6827e-16,
        "Positive electrode LAM constant proportional term [s-1]": 8.3569e-6,
        "Negative electrode LAM constant proportional term [s-1]": 5.8032e-5,
        "EC diffusivity [m2.s-1]": 2e-18,
        "Positive electrode LAM constant exponential term": 2,
        "Negative electrode LAM constant exponential term": 2,
        "Negative electrode active material volume fraction": eps_n_data,
        "Positive electrode active material volume fraction": eps_p_data,
    },
    check_already_exists=False,
)
# all_sumvars_dict = cycle_adaptive_simulation(spm, parameter_values, experiment, save_at_cycles=2)

In [14]:
# print(all_sumvars_dict["cycles evaluated"])
# print(all_sumvars_dict["solution time"])

In [15]:
# fig = plot(all_sumvars_dict,dfe);
# fig.savefig(fig_DIR +'fast_sim_'+cell_no+'_new.png')

In [16]:
# fig = plot1(all_sumvars_dict,dfe);
# fig.savefig(fig_DIR +'fast_sim_'+cell_no+'_cap_new.png')

In [17]:
# fig = plotc(all_sumvars_dict,dfe);

# Parameter Fitting

In [18]:
cycles = np.array(dfe['N'].astype('int'))
cycles=cycles-1
cycles[0]=cycles[0]+1
cycles
dfe['N_mod'] = cycles

In [24]:
param.F.value

96485.33212

In [26]:
def plot_comp(all_sumvars_dict_0,all_sumvars_dict_1,esoh_data):
    esoh_vars = ["x_100", "y_0", "C_n", "C_p", "Capacity [A.h]", "Loss of lithium inventory [%]"]
    esoh_data["Capacity [A.h]"]=esoh_data["Cap"]
    param = pybamm.LithiumIonParameters()
    esoh_data["n_Li"]= 3600/param.F.value*(esoh_data["y_100"]*esoh_data["C_p"]+esoh_data["x_100"]*esoh_data["C_n"])
    esoh_data["Loss of lithium inventory [%]"]=(1-esoh_data["n_Li"]/esoh_data["n_Li"][0])*100
    fig, axes = plt.subplots(3,2,figsize=(7,7))
    for k, name in enumerate(esoh_vars):
        ax = axes.flat[k]
        ax.plot(all_sumvars_dict_1["Cycle number"],all_sumvars_dict_1[name],"bo")
        ax.plot(all_sumvars_dict_0["Cycle number"],all_sumvars_dict_0[name],"ro")
        # ax.plot(all_sumvars_dict_1["Cycle number"],all_sumvars_dict_1[name],"bo")
        ax.plot(esoh_data["N"],esoh_data[name],"kx")
        ax.axvline(x = all_sumvars_dict_0["Cycle number"][-1], color = 'm', label = 'axvline - full height')
        ax.set_title(split_long_string(name))
        if k>3:
            ax.set_xlabel("Cycle number")
    fig.legend(["Pred"]+["Train"] + ["Data"], 
           loc="lower center",bbox_to_anchor=[0.5, -0.05], ncol=1, fontsize=11)
    fig.tight_layout()
    return fig

## List of Initial Conditions

In [20]:
ic = 1
blam_p = [2e-2/3600, 4e-6, 1e-5,1e-6]
blam_n = [2e-1/3600, 3e-5, 2e-5,7e-5]
k_sei= [1.6e-16,1.5e-16,1.7e-16,1.3e-16]

In [21]:
def objective(model, data):
    return np.array(model.loc[data['N_mod']]["Capacity [A.h]"]) - np.array(data["Cap"])

def multi_objective(model, data):
    variables = ["C_n","C_p","x_100","y_0"]
    # variables = ["Capacity [A.h]", "C_n"]
    weights = [1,1,5,5]
    return np.concatenate([
        (np.array(model.loc[data['N_mod']][var]) - np.array(data[var])) * w
        for w,var in zip(weights,variables)
    ]
    )
@timeoutable()
def simulate(x,expt):
    parameter_values = get_parameter_values()
    parameter_values.update(
        {
            "SEI kinetic rate constant [m.s-1]":  x[2]*k_sei[ic],
            "Positive electrode LAM constant proportional term [s-1]": x[0]* blam_p[ic],
            "Negative electrode LAM constant proportional term [s-1]": x[1]*blam_n[ic],
            "EC diffusivity [m2.s-1]": 2e-18,
            "Positive electrode LAM constant exponential term": 2,
            "Negative electrode LAM constant exponential term": 2,
            "Negative electrode active material volume fraction": eps_n_data,
            "Positive electrode active material volume fraction": eps_p_data,
        },
        check_already_exists=False,
    )
    Cn = parameter_values.evaluate(param.C_n_init)
    Cp = parameter_values.evaluate(param.C_p_init)
    eps_n = parameter_values["Negative electrode active material volume fraction"]
    eps_p = parameter_values["Positive electrode active material volume fraction"]
    C_over_eps_n = Cn / eps_n
    C_over_eps_p = Cp / eps_p
    return cycle_adaptive_simulation(spm, parameter_values, expt,save_at_cycles=1)
def prediction_error(x,data,expt,train_len):
    try:
        model = simulate(x,expt,timeout=60)
        out = multi_objective(pd.DataFrame(model), data)
        # print("{}".format(x))
        print(f"Train len = {train_len}, x={x}, norm={np.linalg.norm(out):.4f}")
    # except pybamm.SolverError:
    except:
        out = 10*np.ones_like(np.concatenate([np.array(data['Cap'])]*4))
        print(f"Train len = {train_len}, x={x}, norm={np.linalg.norm(out):.4f}")
        return out
    return out
def train_model(data,expt,train_len):
    timer = pybamm.Timer()
    x0 = np.array([1.0,1.0,1.0])
    # print(prediction_error(x0))
    soln_dfols = dfols.solve(prediction_error, x0,args=(data,expt,train_len), rhoend=1e-3)
    print(timer.time())
    return soln_dfols
def sim_train(dfe,experiment,train_len):
  
    data_train = dfe.iloc[:train_len]
    experiment_train = pybamm.Experiment(
        [
            ("Discharge at "+c_rate_d+dis_set,
            "Rest for 5 min",
            "Charge at "+c_rate_c+" until 4.2V", 
            "Hold at 4.2V until C/50")
        ] *data_train.N.iloc[-1],
        termination="50% capacity",
    #     cccv_handling="ode",
    )
    soln_dfols = train_model(data_train,experiment_train,train_len)
    xsol = soln_dfols.x
    # print(xsol[0]*2e-2/3600)
    # print(xsol[1]*2e-1/3600)
    # print(xsol[2]*1.6e-16)
    y = []
    y.append(train_len)
    y.append(round(xsol[0],4))
    y.append(round(xsol[1],4))
    y.append(round(xsol[2],4))
    print('Final sol = ',str(y))
    # df['obj'][train_len-3]=soln_dfols.f
    # try:
    #     sol_out = simulate([y[1],y[2],y[3]],experiment_train,timeout = 60)
    #     sol_full = simulate([y[1],y[2],y[3]],experiment,timeout = 60)
    # except:
    #     print("Train Len = "+str(train_len)+" failed")
    # y.append(np.linalg.norm(prediction_error([y[1],y[2],y[3]],data_train,experiment_train)))
    # y.append(np.linalg.norm(prediction_error([y[1],y[2],y[3]],dfe,experiment)))
    return y

In [22]:
def f(train_len):
    y = sim_train(dfe,experiment,train_len)
    return y

In [23]:
# yyy = Parallel(n_jobs=2, verbose=11)(delayed(f)(x) for x in [3,4])
with parallel_backend("loky",n_jobs=-1):
    yyy = Parallel()(delayed(f)(x) for x in range(3,len(N)+1))
    # yyy = Parallel()(delayed(f)(x) for x in range(3,5))

In [ ]:
df_x = pd.DataFrame(columns=['train_len','x_0','x_1','x_2','obj','obj_full'], index=range(len(N)+1-3))

In [ ]:
for ii in range(len(yyy)):
    df_x['train_len'][ii] =yyy[ii][0]
    df_x['x_0'][ii] =yyy[ii][1]
    df_x['x_1'][ii] =yyy[ii][2]
    df_x['x_2'][ii] =yyy[ii][3]

In [ ]:
df_x

In [ ]:
sim_des = "ic_"+str(ic)
df_x.to_csv(res_DIR + "fit_train_pred_"+sim_des+".csv")

## Generate figures from solutions

In [ ]:
sim_des = "ic_3"
df_x = pd.read_csv(res_DIR + "fit_train_pred_"+sim_des+".csv").drop(['Unnamed: 0'],axis=1)

In [ ]:
# df_x =df_x.round(4)

In [ ]:
df_x

In [ ]:
def save_fig_obj(train_len):
    data_train = dfe.iloc[:train_len]
    print(train_len)
    x = [df_x['x_0'][train_len-3],df_x['x_1'][train_len-3],df_x['x_2'][train_len-3]]
    experiment_train = pybamm.Experiment(
            [
                ("Discharge at "+c_rate_d+dis_set,
                "Rest for 5 min",
                "Charge at "+c_rate_c+" until 4.2V", 
                "Hold at 4.2V until C/50")
            ] *data_train.N.iloc[-1],
            termination="50% capacity",
        #     cccv_handling="ode",
        )
    sol_out = simulate(x,experiment_train,timeout = 30)
    sol_full = simulate(x,experiment,timeout = 30)
    fig = plot_comp(sol_out,sol_full,dfe);
    fig.suptitle('Train Length = '+str(train_len),fontsize=16,y=1.02)
    fig.savefig(fig_DIR + "fit_train_pred_"+ sim_des+"_"+str(train_len) +"_eSOHC.png",bbox_inches='tight')
    plt.close(fig)
    # df_x['obj'][train_len-3]= np.linalg.norm(prediction_error(x,data_train,experiment_train))
    # df_x['obj_full'][train_len-3] = np.linalg.norm(prediction_error(x,dfe,experiment))

In [ ]:
yyy = Parallel(n_jobs=-1, verbose=100)(delayed(save_fig_obj)(x) for x in range(3,len(N)))

In [ ]:
save_fig_obj(len(N))

In [ ]:
for train_len in [3]:
# for train_len in range(3,len(N)+1):
    data_train = dfe.iloc[:train_len]
    print(train_len)
    x = [df_x['x_0'][train_len-3],df_x['x_1'][train_len-3],df_x['x_2'][train_len-3]]
    experiment_train = pybamm.Experiment(
            [
                ("Discharge at "+c_rate_d+dis_set,
                "Rest for 5 min",
                "Charge at "+c_rate_c+" until 4.2V", 
                "Hold at 4.2V until C/50")
            ] *data_train.N.iloc[-1],
            termination="50% capacity",
        #     cccv_handling="ode",
        )
    sol_out = simulate(x,experiment_train,timeout = 30)
    sol_full = simulate(x,experiment,timeout = 30)
    fig = plot_comp(sol_out,sol_full,dfe);
    fig.suptitle('Train Length = '+str(train_len),fontsize=16,y=1.02)
    fig.savefig(fig_DIR + "fit_train_pred_"+ sim_des+"_"+str(train_len) +"_eSOHC.png",bbox_inches='tight')
    plt.close(fig)
    # df_x['obj'][train_len-3]= np.linalg.norm(prediction_error(x,data_train,experiment_train))
    # df_x['obj_full'][train_len-3] = np.linalg.norm(prediction_error(x,dfe,experiment))

In [ ]:
df_x['obj'] = df_x['obj'].astype(float)
df_x['obj_full'] = df_x['obj_full'].astype(float)
df_x =df_x.round(4)
df_x

In [ ]:
df_x.to_csv(res_DIR + "fit_train_pred_"+sim_des+".csv")

## Plot Tuned Parameters

In [ ]:
for ic in range(0,2):
    print(ic)

In [ ]:
for ic in range(0,3):
    sim_des = "ic_"+str(ic)
    df_x = pd.read_csv(res_DIR + "fit_train_pred_"+sim_des+".csv").drop(['Unnamed: 0'],axis=1)
    fig, axes = plt.subplots(3,1,figsize=(7,7))
    ax1 = axes.flat[0]
    ax1.plot(df_x['train_len'],df_x['x_0']*k_sei[ic],'x-')
    ax1.set_title(r'$\beta_{LAM}^+$')
    ax2 = axes.flat[1]
    ax2.plot(df_x['train_len'],df_x['x_1']*blam_p[ic],'x-')
    ax2.set_title(r'$\beta_{LAM}^-$')
    ax3 = axes.flat[2]
    ax3.plot(df_x['train_len'],df_x['x_2']*blam_n[ic],'x-')
    ax3.set_title(r'$\kappa_{SEI}$')
    ax3.set_xlabel('Training Length')
fig.legend(["ic = 0","ic = 1","ic = 2"])
fig.tight_layout()    
fig.savefig(fig_DIR +'trained_parameters_'+sim_des+'.png')

In [ ]:
fig, axes = plt.subplots(3,1,figsize=(7,7))
ax1 = axes.flat[0]
ax1.plot(df_x['train_len'],df_x['x_0']*k_sei[ic],'x-')
ax1.set_title(r'$\beta_{LAM}^+$')
ax2 = axes.flat[1]
ax2.plot(df_x['train_len'],df_x['x_1']*blam_p[ic],'x-')
ax2.set_title(r'$\beta_{LAM}^-$')
ax3 = axes.flat[2]
ax3.plot(df_x['train_len'],df_x['x_2']*blam_n[ic],'x-')
ax3.set_title(r'$\kappa_{SEI}$')
ax3.set_xlabel('Training Length')
fig.tight_layout()
fig.savefig(fig_DIR +'trained_parameters_'+sim_des+'.png')